In [1]:
!python -V

Python 3.9.12


In [3]:
# import library 
import pandas as pd 
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow 
mlflow.set_tracking_uri("sqlite:///mflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/04 22:38:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/04 22:38:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1746398281006, experiment_id='1', last_update_time=1746398281006, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

### Put all the proccessing into a function

In [2]:
def read_dataframe(filename):
    "Write a function to read and preprocessing data"
    # Read the dataset
    df_taxi = pd.read_parquet(filename)

    # Adjust dropoff & pickup to pandas datetime 
    df_taxi['lpep_pickup_datetime'] = pd.to_datetime(df_taxi.lpep_pickup_datetime)
    df_taxi['lpep_dropoff_datetime'] = pd.to_datetime(df_taxi.lpep_dropoff_datetime)
    
    # Calculate the duration (drop_off -  pick_up)
    df_taxi['duration'] = df_taxi.lpep_dropoff_datetime - df_taxi.lpep_pickup_datetime
    
    # Adjust the duration in minutes for prediction 
    df_taxi['duration_minutes'] = df_taxi['duration'].dt.total_seconds() / 60

    # Since there are a lot of duration less than 1 minutes. We filter only duration between 1 minutes to 99% percentile
    df_taxi = df_taxi[(df_taxi['duration_minutes'] >= 1) & (df_taxi['duration_minutes'] <= 60)]

    # Feature Engineering 
    categorical_variables = ['PULocationID', 'DOLocationID']
    numerical_variables = ['trip_distance']

    # Convert it into "str"
    df_taxi[categorical_variables] = df_taxi[categorical_variables].astype(str)
    
    return df_taxi

In [3]:
df_train = read_dataframe('../00-Dataset/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../00-Dataset/green_tripdata_2021-02.parquet')

In [4]:
len(df_train), len(df_val)

(73908, 61921)

### Create the training pipeline 

In [5]:
# Feature Engineering 
categorical_variables = ['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [6]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [7]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

10.47390313604031

### Try with Lasso Regression

In [74]:
# Train the model 
lr = Lasso(alpha= 0.0001) 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

10.47054920225702

### Try To Combine the input features 

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
# Feature Engineering 
categorical_variables = ['PU_DO']  #['PULocationID', 'DOLocationID']
numerical_variables = ['trip_distance']

# Vectorizer the training variables 
dv = DictVectorizer()

# Convert it into dictionary 
train_dicts = df_train[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

# Create the validation set 
val_dicts = df_val[categorical_variables + numerical_variables].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [10]:
# Setup the Prediction_Variables 
predictor = 'duration_minutes'
y_train = df_train[predictor].values
y_val = df_val[predictor].values

In [11]:
# Train the model 
lr = LinearRegression() 
lr.fit(X_train, y_train)

# Make a prediction 
y_pred = lr.predict(X_val)

# Calculate the performance (RMSE)
mean_squared_error(y_val, y_pred, squared = False) 

7.479586896299878